In [1]:
import datetime
from dotenv import load_dotenv
load_dotenv('semantic-kernel/python/.env')
import os
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
model = "gpt-4-1106-preview"

# With semantic kernel
* Note semantic-kernel requires openai == 0.28, not compatible with v1.0 yet.

In [2]:
import semantic_kernel as sk
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from sk_custom_skills.Order import Order
from semantic_kernel.planning import StepwisePlanner
from semantic_kernel.planning.stepwise_planner.stepwise_planner_config import StepwisePlannerConfig

# set up kernel
kernel = sk.Kernel()
kernel.add_chat_service("gpt-4", OpenAIChatCompletion(model, OPENAI_API_KEY))

# Add custom skills
skills_directory = "sk_custom_skills"
semantic_siklls = kernel.import_semantic_skill_from_directory(skills_directory, "semantic_skills")
custom_plugin = kernel.import_skill(Order(), skill_name="order")

# function to add chat history to context
def update_ask(user_input, chat_history, language="English"):
    return f"""
        Answer the following questions from EC customer based on the past chat histroy in {language}.
        Return policy : accept returns within 30 days from the delivery date with full refund without asking reason.
        #### Chat History ####
        {'; '.join(chat_history)}
        #### User Input ####
        {user_input} """

In [3]:
# setup planner
from semantic_kernel.planning import StepwisePlanner
from semantic_kernel.planning.stepwise_planner.stepwise_planner_config import StepwisePlannerConfig
planner = StepwisePlanner(
    kernel, StepwisePlannerConfig(max_iterations=10, min_iteration_time_ms=1000)
)

In [4]:
#%% execute with user input
chat_history = []
user_input = "What is the status of my order for B? order_id is 2."
ask = update_ask(user_input, chat_history)

stepwise_plan = planner.create_plan(goal=ask)
result = await stepwise_plan.invoke_async()
print(result)

# update chat history
chat_history.append("User: " + user_input + "\n")
chat_history.append("Assistant: " + result.variables['input'])

The status of your order for item B with order_id 2 is completed. It was delivered on November 23, 2023. If you need further assistance or have any other questions, feel free to ask!


In [5]:
# to check what is happening behind the scene
for index, step in enumerate(stepwise_plan._steps):
    print("Step:", index)
    print("Description:",step.description)
    print("Function:", step.skill_name + "." + step._function.name)
    if len(step._outputs) > 0:
        print( "  Output:\n", str.replace(result[step._outputs[0]],"\n", "\n  "))

Step: 0
Description: Execute a plan
Function: StepwisePlanner.ExecutePlan
  Output:
 This was my previous work (but they haven't seen any of it! They only see what I return as final answer):
  [THOUGHT]
  The customer is asking for the status of their order for item B with the order_id of 2. I need to use the order.get_order_status function to retrieve this information.
  [ACTION]
  {"action": "order.get_order_status", "action_variables": {"input": "2"}}
  [OBSERVATION]
  {'item_name': 'B', 'order_date': '2023-11-22', 'delivery_status': 'completed', 'delivery_date': '2023-11-23'}
  [THOUGHT]
  None


In [6]:
# follow up question
user_input = "Can I return this item?"
ask = update_ask(user_input, chat_history)
stepwise_plan = planner.create_plan(goal=ask)
result = await stepwise_plan.invoke_async()
print(result)

Yes, you can return the item. Our return policy allows you to return items within 30 days from the delivery date for a full refund without any need to provide a reason. Since your item B was delivered on November 23, 2023, and today is December 1, 2023, you are within the 30-day window to initiate a return.


In [7]:
# to check what is happening behind the scene
for index, step in enumerate(stepwise_plan._steps):
    print("Step:", index)
    print("Description:",step.description)
    print("Function:", step.skill_name + "." + step._function.name)
    if len(step._outputs) > 0:
        print( "  Output:\n", str.replace(result[step._outputs[0]],"\n", "\n  "))

Step: 0
Description: Execute a plan
Function: StepwisePlanner.ExecutePlan
  Output:
 This was my previous work (but they haven't seen any of it! They only see what I return as final answer):
  [THOUGHT]
  The user is asking if they can return an item, presumably item B, which was mentioned in the chat history. To answer this question, I need to know today's date to calculate if the return is within the 30-day return policy window from the delivery date, which was November 23, 2023. I will use the "order.current_date" function to get today's date.
  [ACTION]
  {"action": "order.current_date", "action_variables": {"input": ""}}
  [OBSERVATION]
  2023-12-01
  [THOUGHT]
  None


In [8]:
# same but ask to repply in French
user_input = "Can I return this item?"
ask = update_ask(user_input, chat_history, language="French")
stepwise_plan = planner.create_plan(goal=ask)
result = await stepwise_plan.invoke_async()
print(result)

Oui, vous pouvez retourner l'article. Il a été livré le 23 novembre 2023 et comme aujourd'hui nous sommes le 1er décembre 2023, vous êtes dans le délai de retour de 30 jours.


In [9]:
# to check what is happening behind the scene
for index, step in enumerate(stepwise_plan._steps):
    print("Step:", index)
    print("Description:",step.description)
    print("Function:", step.skill_name + "." + step._function.name)
    if len(step._outputs) > 0:
        print( "  Output:\n", str.replace(result[step._outputs[0]],"\n", "\n  "))

Step: 0
Description: Execute a plan
Function: StepwisePlanner.ExecutePlan
  Output:
 This was my previous work (but they haven't seen any of it! They only see what I return as final answer):
  [THOUGHT]
  The user is asking if they can return an item. Based on the return policy provided, the customer can return items within 30 days from the delivery date with a full refund without providing a reason. To answer the user's question, I need to determine the current date and compare it with the delivery date of November 23, 2023, to see if it falls within the 30-day return window. I will use the "order.current_date" function to get today's date.
  [ACTION]
  {"action": "order.current_date", "action_variables": {"input": ""}}
  [OBSERVATION]
  2023-12-01
  [THOUGHT]
  The current date is December 1, 2023, as observed from the action taken. The delivery date of the item was November 23, 2023. To determine if the item can be returned, I need to calculate if December 1, 2023, is within 30 days

### Ref: 
* Documentation https://learn.microsoft.com/en-us/semantic-kernel/
* Github https://github.com/microsoft/semantic-kernel